<a href="https://colab.research.google.com/github/abhranil-datascience/PrescriptionAdherenceUsecase/blob/master/PrescriptionAdherence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
######################### Suppress Warnings #######################################

#import warnings
#warnings.filterwarnings('ignore')

############################## Mount Drive ########################################

from google.colab import drive
drive.mount('/content/gdrive')

############################## Change Directory ###################################

import os
os.chdir('/content/gdrive/My Drive/AbzoobaTest/Dataset/')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
######################################### Import Datasets ###########################################################
import pandas as pd
TrainDataFrame=pd.read_csv('Training Data.csv')
TestDataFrame=pd.read_csv('Test Data.csv')

In [0]:
######################################## Look At Training Data Info #################################################
TrainDataFrame.info()
## ==================> No null object

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180212 entries, 0 to 180211
Data columns (total 11 columns):
patient_id             180212 non-null int64
Age                    180212 non-null int64
Gender                 180212 non-null object
Prescription_period    180212 non-null int64
Diabetes               180212 non-null int64
Alcoholism             180212 non-null int64
HyperTension           180212 non-null int64
Smokes                 180212 non-null int64
Tuberculosis           180212 non-null int64
Sms_Reminder           180212 non-null int64
Adherence              180212 non-null object
dtypes: int64(9), object(2)
memory usage: 15.1+ MB


In [0]:
######################################## Look At Test Data Info #################################################
TestDataFrame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119788 entries, 0 to 119787
Data columns (total 10 columns):
patient_id             119788 non-null int64
Age                    119788 non-null int64
Gender                 119788 non-null object
Prescription_period    119788 non-null int64
Diabetes               119788 non-null int64
Alcoholism             119788 non-null int64
HyperTension           119788 non-null int64
Smokes                 119788 non-null int64
Tuberculosis           119788 non-null int64
Sms_Reminder           119788 non-null int64
dtypes: int64(9), object(1)
memory usage: 9.1+ MB


In [0]:
#################################### Check Class Imbalance in Training Set ##########################################
TrainDataFrame['Adherence'].value_counts()
## ====================> Class Imbalance is there 
## No: 125822
## Yes: 54390
## No to Yes ratio: 125822/54390=2.31
## Yes to be labeled as 1 and No to be labeled as 0

No     125822
Yes     54390
Name: Adherence, dtype: int64

In [0]:
################################## Understand Data Profile #########################################
import pandas_profiling
pandas_profiling.ProfileReport(TrainDataFrame)
## Things to note 
## 1. There are 6167 records with Age as 0.
## 2. Only 2.5% records have alcoholism value as 1.
## 3. Only 7.9% records have Diabetes value as 1.
## 4. 66.8% are females and 33.2% are males.
## 5. Only 21.7% records have hypertension value as 1
## 6. Only 5.3% records have smoke value as 1
## 7. Only 0.3% records have sms reminder value as 2, 42.9% records have  value 0 and 56.9% value has 1
## 8. Only 61 records (almost 0%) records have tuberculosis.
## 9. Strongest Correlation exist between Age and Hypertension
## 10. Correlation exist between Age and Diabetes
## 11. Correlation exist between Diabetes and Hypertension
## 12. Correlation exist between Alcoholism and Smoke


Number of variables,11
Number of observations,180212
Total Missing (%),0.0%
Total size in memory,15.1 MiB
Average record size in memory,88.0 B
Numeric,4
Categorical,2
Boolean,5
Date,0
Text (Unique),0
Rejected,0


In [0]:
##################################### Generate Derived Variables #######################################################
def disease_count(row):
  current_patient_diabetes=row['Diabetes']
  current_patient_hypertension=row['HyperTension']
  current_patient_tuberculosis=row['Tuberculosis']
  num_of_diseases=current_patient_diabetes+current_patient_hypertension+current_patient_tuberculosis
  return num_of_diseases
DiseaseCountSeries=TrainDataFrame.apply(func=disease_count,axis='columns')
def smoke_alcohol(row):
  current_patient_smoke=row['Smokes']
  current_patient_drinks=row['Alcoholism']
  if current_patient_smoke==0 and current_patient_drinks==0:
    return 'N'
  elif current_patient_smoke==1 and current_patient_drinks==0:
    return 'S'
  elif current_patient_smoke==0 and current_patient_drinks==1:
    return 'D'
  else:
    return 'B'
AddictionSeries=TrainDataFrame.apply(func=smoke_alcohol,axis='columns')
def age_cat(row):
  current_patient_age=row['Age']
  if current_patient_age >=0 and current_patient_age <=10:
    return 'a'
  elif current_patient_age >=11 and current_patient_age <=20:
    return 'b'
  elif current_patient_age >=21 and current_patient_age <=30:
    return 'c'
  elif current_patient_age >=31 and current_patient_age <=40:
    return 'd'
  elif current_patient_age >=41 and current_patient_age <=50:
    return 'e'
  elif current_patient_age >=51 and current_patient_age <=60:
    return 'f'
  elif current_patient_age >=61 and current_patient_age <=70:
    return 'g'
  elif current_patient_age >=71 and current_patient_age <=80:
    return 'h'
  elif current_patient_age >=91 and current_patient_age <=100:
    return 'i'
  else:
    return 'j'
AgeCategorySeries=TrainDataFrame.apply(func=age_cat,axis='columns')
def SMSReminderCategory(row):
  if row['Sms_Reminder']==0:
    return 'N'
  else:
    return 'Y'
SMSReminder=TrainDataFrame.apply(func=SMSReminderCategory,axis='columns')
def DiseaseCategory(row):
  IsDiabetes=row['Diabetes']
  IsHypertension=row['HyperTension']
  IsTubeculosis=row['Tuberculosis']
  if IsDiabetes==0 and IsHypertension==0 and IsTubeculosis==0:
    return 'N'
  elif IsDiabetes==0 and IsHypertension==0 and IsTubeculosis==1:
    return 'T'
  elif IsDiabetes==0 and IsHypertension==1 and IsTubeculosis==0:
    return 'H'
  elif IsDiabetes==0 and IsHypertension==1 and IsTubeculosis==1:
    return 'HT'
  elif IsDiabetes==1 and IsHypertension==0 and IsTubeculosis==0:
    return 'D'
  elif IsDiabetes==1 and IsHypertension==0 and IsTubeculosis==1:
    return 'DT'
  elif IsDiabetes==1 and IsHypertension==1 and IsTubeculosis==0:
    return 'DH'
  elif IsDiabetes==1 and IsHypertension==1 and IsTubeculosis==1:
    return 'DHT'
  else:
    return 'NA'
DiseaseCategorySeries=TrainDataFrame.apply(func=DiseaseCategory,axis='columns')

In [0]:
######################################### Add Derived Column To TrainDataFrame #######################################################
TrainDataFrame['DiseaseCount']=DiseaseCountSeries
TrainDataFrame['Addiction']=AddictionSeries
TrainDataFrame['AgeCategory']=AgeCategorySeries
TrainDataFrame['SMSReminder']=SMSReminder
TrainDataFrame['DiseaseCategory']=DiseaseCategorySeries
TrainDataFrame.head(5)

,patient_id,Age,Gender,Prescription_period,Diabetes,Alcoholism,HyperTension,Smokes,Tuberculosis,Sms_Reminder,Adherence,DiseaseCount,Addiction,AgeCategory,SMSReminder,DiseaseCategory
0,1,19,M,7,0,0,0,0,0,0,No,0,N,b,N,N
1,2,24,F,59,0,0,0,0,0,0,No,0,N,c,N,N
2,3,4,F,43,0,0,0,0,0,0,No,0,N,a,N,N
3,4,38,M,66,0,0,0,0,0,1,No,0,N,d,Y,N
4,5,46,F,98,0,0,0,0,0,1,No,0,N,e,Y,N


In [0]:
############################################## Drop Unnecessary Columns ##########################################################
TrainDataFrameAnalyzed=TrainDataFrame.drop(['Age','Alcoholism','Smokes','Sms_Reminder','Diabetes','HyperTension','Tuberculosis'],axis=1)
TrainDataFrameAnalyzed=TrainDataFrameAnalyzed[['patient_id','AgeCategory','Gender','Addiction','DiseaseCategory','DiseaseCount','Prescription_period','SMSReminder','Adherence']]
TrainDataFrameAnalyzed.head(10)

,patient_id,AgeCategory,Gender,Addiction,DiseaseCategory,DiseaseCount,Prescription_period,SMSReminder,Adherence
0,1,b,M,N,N,0,7,N,No
1,2,c,F,N,N,0,59,N,No
2,3,a,F,N,N,0,43,N,No
3,4,d,M,N,N,0,66,Y,No
4,5,e,F,N,N,0,98,Y,No
5,6,b,F,N,N,0,73,N,No
6,7,f,F,N,DH,2,102,Y,No
7,8,f,F,N,H,1,96,Y,No
8,9,h,M,N,H,1,89,N,No
9,10,e,F,N,N,0,97,Y,No


In [0]:
################################################### Do the Same For Test Dataset ###################################################
DiseaseCountSeries=TestDataFrame.apply(func=disease_count,axis='columns')
AddictionSeries=TestDataFrame.apply(func=smoke_alcohol,axis='columns')
AgeCategorySeries=TestDataFrame.apply(func=age_cat,axis='columns')
SMSReminder=TestDataFrame.apply(func=SMSReminderCategory,axis='columns')

In [0]:
######################################### Add Derived Column To TestDataFrame #######################################################
TestDataFrame['DiseaseCount']=DiseaseCountSeries
TestDataFrame['Addiction']=AddictionSeries
TestDataFrame['AgeCategory']=AgeCategorySeries
TestDataFrame['SMSReminder']=SMSReminder
TestDataFrame['DiseaseCategory']=DiseaseCategorySeries
TestDataFrame.head(5)

,patient_id,Age,Gender,Prescription_period,Diabetes,Alcoholism,HyperTension,Smokes,Tuberculosis,Sms_Reminder,DiseaseCount,Addiction,AgeCategory,SMSReminder,DiseaseCategory
0,1,5,M,28,0,0,0,0,0,1,0,N,a,Y,N
1,2,62,F,9,1,0,1,0,0,0,2,N,g,N,N
2,3,4,F,73,0,0,0,0,0,1,0,N,a,Y,N
3,4,33,M,117,0,0,0,0,0,0,0,N,d,N,N
4,5,38,M,8,0,0,0,0,0,1,0,N,d,Y,N


In [0]:
############################################## Drop Unnecessary Columns ##########################################################
TestDataFrameAnalyzed=TestDataFrame.drop(['Age','Alcoholism','Smokes','Sms_Reminder','Diabetes','HyperTension','Tuberculosis'],axis=1)
TestDataFrameAnalyzed=TestDataFrameAnalyzed[['patient_id','AgeCategory','Gender','Addiction','DiseaseCategory','DiseaseCount','Prescription_period','SMSReminder']]
TestDataFrameAnalyzed.head(10)

,patient_id,AgeCategory,Gender,Addiction,DiseaseCategory,DiseaseCount,Prescription_period,SMSReminder
0,1,a,M,N,N,0,28,Y
1,2,g,F,N,N,2,9,N
2,3,a,F,N,N,0,73,Y
3,4,d,M,N,N,0,117,N
4,5,d,M,N,N,0,8,Y
5,6,g,F,N,N,0,103,Y
6,7,f,M,N,DH,0,7,N
7,8,a,M,N,H,0,48,Y
8,9,a,M,N,H,0,86,N
9,10,d,F,N,N,0,59,N


In [0]:
############################################## Label Encode And One Hot Encode TrainDataset ##############################################################
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
LE_AgeCategory=LabelEncoder()
LE_Gender=LabelEncoder()
LE_Addiction=LabelEncoder()
LE_DiseaseCategory=LabelEncoder()
LE_SMSReminder=LabelEncoder()
XTrain=TrainDataFrameAnalyzed.iloc[:,1:8].values
YTrain=TrainDataFrameAnalyzed.iloc[:,8].values
XTrain[0:5,:]

array([['b', 'M', 'N', 'N', 0, 7, 'N'],
       ['c', 'F', 'N', 'N', 0, 59, 'N'],
       ['a', 'F', 'N', 'N', 0, 43, 'N'],
       ['d', 'M', 'N', 'N', 0, 66, 'Y'],
       ['e', 'F', 'N', 'N', 0, 98, 'Y']], dtype=object)

In [0]:
XTrain[:,0]=LE_AgeCategory.fit_transform(XTrain[:,0])
XTrain[:,1]=LE_Gender.fit_transform(XTrain[:,1])
XTrain[:,2]=LE_Addiction.fit_transform(XTrain[:,2])
XTrain[:,3]=LE_DiseaseCategory.fit_transform(XTrain[:,3])
XTrain[:,6]=LE_SMSReminder.fit_transform(XTrain[:,6])
XTrain[0:5,:]

array([[1, 1, 2, 6, 0, 7, 0],
       [2, 0, 2, 6, 0, 59, 0],
       [0, 0, 2, 6, 0, 43, 0],
       [3, 1, 2, 6, 0, 66, 1],
       [4, 0, 2, 6, 0, 98, 1]], dtype=object)

In [0]:
ohe_AgeCategory=OneHotEncoder(categorical_features = [0])
XTrainAgeEnocded=ohe_AgeCategory.fit_transform(XTrain).toarray()
XTrainAgeEnocded=XTrainAgeEnocded[:,1:]
XTrainAgeEnocded[0:5,:]

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  2.,  6.,  0.,
         7.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  6.,  0.,
        59.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  6.,  0.,
        43.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  2.,  6.,  0.,
        66.,  1.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  6.,  0.,
        98.,  1.]])

In [0]:
ohe_Addiction=OneHotEncoder(categorical_features = [10])
XTrainAddictionEncoded=ohe_Addiction.fit_transform(XTrainAgeEnocded).toarray()
XTrainAddictionEncoded=XTrainAddictionEncoded[:,1:]
XTrainAddictionEncoded[0:4,:]

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


array([[ 0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
         6.,  0.,  7.,  0.],
       [ 0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         6.,  0., 59.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         6.,  0., 43.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
         6.,  0., 66.,  1.]])

In [0]:
ohe_DiseaseCategory=OneHotEncoder(categorical_features = [13])
XTrainDiseaseCategoryEncoded=ohe_DiseaseCategory.fit_transform(XTrainAddictionEncoded).toarray()
XTrainDiseaseCategoryEncoded=XTrainDiseaseCategoryEncoded[:,1:]
XTrainDiseaseCategoryEncoded[0:4,:]

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


array([[ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  7.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 59.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 43.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,
         0.,  0.,  0.,  0.,  0.,  0.,  1.,  0., 66.,  1.]])

In [0]:
############################################## Label Encode And One Hot Encode TestDataset ##############################################################
XTest=TestDataFrameAnalyzed.iloc[:,1:8].values
XTest[0:5,:]

array([['a', 'M', 'N', 'N', 0, 28, 'Y'],
       ['g', 'F', 'N', 'N', 2, 9, 'N'],
       ['a', 'F', 'N', 'N', 0, 73, 'Y'],
       ['d', 'M', 'N', 'N', 0, 117, 'N'],
       ['d', 'M', 'N', 'N', 0, 8, 'Y']], dtype=object)

In [0]:
XTest[:,0]=LE_AgeCategory.transform(XTest[:,0])
XTest[:,1]=LE_Gender.transform(XTest[:,1])
XTest[:,2]=LE_Addiction.transform(XTest[:,2])
XTest[:,3]=LE_DiseaseCategory.transform(XTest[:,3])
XTest[:,6]=LE_SMSReminder.transform(XTest[:,6])
XTest[0:5,:]

array([[0, 1, 2, 6, 0, 28, 1],
       [6, 0, 2, 6, 2, 9, 0],
       [0, 0, 2, 6, 0, 73, 1],
       [3, 1, 2, 6, 0, 117, 0],
       [3, 1, 2, 6, 0, 8, 1]], dtype=object)

In [0]:
XTestAgeEnocded=ohe_AgeCategory.fit_transform(XTest).toarray()
XTestAgeEnocded=XTestAgeEnocded[:,1:]
XTestAgeEnocded[0:5,:]

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


array([[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   1.,   2.,
          6.,   0.,  28.,   1.],
       [  0.,   0.,   0.,   0.,   0.,   1.,   0.,   0.,   0.,   0.,   2.,
          6.,   2.,   9.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   2.,
          6.,   0.,  73.,   1.],
       [  0.,   0.,   1.,   0.,   0.,   0.,   0.,   0.,   0.,   1.,   2.,
          6.,   0., 117.,   0.],
       [  0.,   0.,   1.,   0.,   0.,   0.,   0.,   0.,   0.,   1.,   2.,
          6.,   0.,   8.,   1.]])

In [0]:
XTestAddictionEncoded=ohe_Addiction.transform(XTestAgeEnocded).toarray()
XTestAddictionEncoded=XTestAddictionEncoded[:,1:]
XTestAddictionEncoded[0:4,:]

array([[  0.,   1.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   1.,   6.,   0.,  28.,   1.],
       [  0.,   1.,   0.,   0.,   0.,   0.,   0.,   0.,   1.,   0.,   0.,
          0.,   0.,   6.,   2.,   9.,   0.],
       [  0.,   1.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   6.,   0.,  73.,   1.],
       [  0.,   1.,   0.,   0.,   0.,   1.,   0.,   0.,   0.,   0.,   0.,
          0.,   1.,   6.,   0., 117.,   0.]])

In [0]:
XTestDiseaseCategoryEncoded=ohe_DiseaseCategory.transform(XTestAddictionEncoded).toarray()
XTestDiseaseCategoryEncoded=XTestDiseaseCategoryEncoded[:,1:]
XTestDiseaseCategoryEncoded[0:4,:]

array([[  0.,   0.,   0.,   0.,   0.,   1.,   0.,   0.,   1.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   1.,   0.,  28.,
          1.],
       [  0.,   0.,   0.,   0.,   0.,   1.,   0.,   0.,   1.,   0.,   0.,
          0.,   0.,   0.,   0.,   1.,   0.,   0.,   0.,   0.,   2.,   9.,
          0.],
       [  0.,   0.,   0.,   0.,   0.,   1.,   0.,   0.,   1.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  73.,
          1.],
       [  0.,   0.,   0.,   0.,   0.,   1.,   0.,   0.,   1.,   0.,   0.,
          0.,   1.,   0.,   0.,   0.,   0.,   0.,   0.,   1.,   0., 117.,
          0.]])

In [0]:
#################################################### Convert Labels in Training Set to 0 and 1 ##########################################################
YTrain[YTrain=='Yes']=1
YTrain[YTrain=='No']=0
YTrain[0:15]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], dtype=object)

In [0]:
#################################################### Scale Dataset #############################################################
from sklearn.preprocessing import StandardScaler
XScaler=StandardScaler()
XTrainScaled=XScaler.fit_transform(XTrainDiseaseCategoryEncoded)
XTestScaled=XScaler.transform(XTestDiseaseCategoryEncoded)
print(str(XTrainScaled.shape)+" || "+str(XTestScaled.shape))

(180212, 23) || (119788, 23)


In [0]:
######################################## Train Test Split #########################################################
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(XTrainScaled,YTrain,test_size=0.15,stratify=YTrain)
print("Number of 0 in train set: "+str(len(y_train[y_train==0])))
print("Number of 1 in train set: "+str(len(y_train[y_train==1])))
print("Number of 0 in val set: "+str(len(y_val[y_val==0])))
print("Number of 1 in val set: "+str(len(y_val[y_val==1])))
y_train=y_train.astype('int')
y_val=y_val.astype('int')

Number of 0 in train set: 106949
Number of 1 in train set: 46231
Number of 0 in val set: 18873
Number of 1 in val set: 8159


In [0]:
################################# Check Random Forest Fit Without Feature Selection And Tuning ################################################
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,f1_score
class_weight={0:1,1:2.313}
RF=RandomForestClassifier(class_weight=class_weight)
RF.fit(X_train,y_train)
YPred=RF.predict(X_val)
confusion_matrix(y_val,YPred)

array([[16691,  2182],
       [  901,  7258]])

In [0]:
f1_score(y_val,YPred)

0.824819592022274

In [0]:
"""
############################################ Hyperparameter Tuning in Random Forest ############################################################
from sklearn.model_selection import GridSearchCV
RF=RandomForestClassifier(class_weight=class_weight)
param_grid = {
    #'bootstrap': [True],
    #'max_depth': [1,10,20,30],
    #'max_features': [2, 3],
    #'min_samples_leaf': [3, 4, 5],
    #'min_samples_split': [8, 10, 12],
    'n_estimators': [180,200,220]
}
grid_search = GridSearchCV(estimator = RF, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2,scoring='f1')
grid_search.fit(XTrainScaled, YTrain.astype(int))
grid_search.best_params_

######### OUTPUT ###############
Fitting 3 folds for each of 3 candidates, totalling 9 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:  3.3min finished
{'n_estimators': 220}
"""

Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:  2.6min finished


{'n_estimators': 180}

In [0]:
"""
############################################ Hyperparameter Tuning in Random Forest ############################################################
from sklearn.model_selection import GridSearchCV
RF=RandomForestClassifier(class_weight=class_weight)
param_grid = {
    #'bootstrap': [True],
    #'max_depth': [1,10,20,30],
    #'max_features': [2, 3],
    #'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [50,60,70,80],
    'n_estimators': [180]
}
grid_search = GridSearchCV(estimator = RF, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2,scoring='f1')
grid_search.fit(XTrainScaled, YTrain.astype(int))
grid_search.best_params_
######### OUTPUT ###############
Fitting 3 folds for each of 4 candidates, totalling 12 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  4.3min finished
{'min_samples_split': 70, 'n_estimators': 200}
"""

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  2.8min finished


{'min_samples_split': 80, 'n_estimators': 180}

In [0]:
"""
############################################ Hyperparameter Tuning in Random Forest ############################################################
from sklearn.model_selection import GridSearchCV
RF=RandomForestClassifier(class_weight=class_weight)
param_grid = {
    #'bootstrap': [True],
    #'max_depth': [1,10,20,30],
    #'max_features': [2, 3],
    'min_samples_leaf': [5, 10, 15],
    'min_samples_split': [80],
    'n_estimators': [180]
}
grid_search = GridSearchCV(estimator = RF, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2,scoring='f1')
grid_search.fit(XTrainScaled, YTrain.astype(int))
grid_search.best_params_
######### OUTPUT ###############
Fitting 3 folds for each of 3 candidates, totalling 9 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:  2.6min finished
{'min_samples_leaf': 10, 'min_samples_split': 70, 'n_estimators': 220}
"""

Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:  1.9min finished


{'min_samples_leaf': 10, 'min_samples_split': 80, 'n_estimators': 180}

In [0]:
"""
############################################ Hyperparameter Tuning in Random Forest ############################################################
from sklearn.model_selection import GridSearchCV
RF=RandomForestClassifier(class_weight=class_weight)
param_grid = {
    #'bootstrap': [True],
    'max_depth': [1,5,10],
    'min_samples_leaf': [10],
    'min_samples_split': [80],
    'n_estimators': [180]
}
grid_search = GridSearchCV(estimator = RF, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2,scoring='f1')
grid_search.fit(XTrainScaled, YTrain.astype(int))
grid_search.best_params_
######### OUTPUT ###############
Fitting 3 folds for each of 3 candidates, totalling 9 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:  1.7min finished
{'max_depth': 15,
 'min_samples_leaf': 10,
 'min_samples_split': 60,
 'n_estimators': 200}
"""

Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:   49.2s finished


{'max_depth': 1,
 'min_samples_leaf': 10,
 'min_samples_split': 80,
 'n_estimators': 180}

In [0]:
####################################### Check If tuning did any improvement ##############################################################
RF=RandomForestClassifier(class_weight=class_weight,max_depth=1,min_samples_leaf=10,min_samples_split=80,n_estimators=180)
RF.fit(X_train,y_train)
YPred=RF.predict(X_val)
confusion_matrix(y_val,YPred)

array([[16719,  2154],
       [  771,  7388]])

In [0]:
f1_score(y_val,YPred)

0.8347550985820009

In [0]:
############################################# Feature Selection Using Random Forest #################################################################
RF=RandomForestClassifier(class_weight=class_weight,max_depth=1,min_samples_leaf=10,min_samples_split=80,n_estimators=180)
YTrain=YTrain.astype(int)
RF.fit(XTrainScaled,YTrain)
import numpy as np
np.around(RF.feature_importances_, decimals=4)

array([0.0278, 0.    , 0.    , 0.0889, 0.    , 0.1222, 0.    , 0.0056,
       0.0167, 0.0056, 0.0389, 0.05  , 0.0111, 0.    , 0.0611, 0.1333,
       0.0389, 0.0056, 0.0278, 0.0111, 0.1722, 0.1833, 0.    ])

In [0]:
b=[0.0278, 0.    , 0.    , 0.0889, 0.    , 0.1222, 0.    , 0.0056,
       0.0167, 0.0056, 0.0389, 0.05  , 0.0111, 0.    , 0.0611, 0.1333,
       0.0389, 0.0056, 0.0278, 0.0111, 0.1722, 0.1833, 0.    ]
sorted(b,reverse=True)

[0.1833,
 0.1722,
 0.1333,
 0.1222,
 0.0889,
 0.0611,
 0.05,
 0.0389,
 0.0389,
 0.0278,
 0.0278,
 0.0167,
 0.0111,
 0.0111,
 0.0056,
 0.0056,
 0.0056,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [0]:
RF=RandomForestClassifier(class_weight=class_weight,max_depth=1,min_samples_leaf=10,min_samples_split=70,n_estimators=220)
RF.fit(X_train[:,[20,21]],y_train)
YPred=RF.predict(X_val[:,[20,21]])
confusion_matrix(y_val,YPred)

array([[ 5635, 13238],
       [ 1793,  6366]])

In [0]:
f1_score(y_val,YPred)

0.8347550985820009

In [0]:
#################################### kFold Cross Validation ################################################
from sklearn.model_selection import StratifiedKFold
kf=StratifiedKFold(n_splits=12,shuffle=True)
f1_score_list=[]
for train_index,test_index in kf.split(X=XTrainScaled,y=YTrain.astype(int)):
  XTrainKF,YTrainKF=XTrainScaled[train_index,:],YTrain[train_index]
  XValKF,YValKF=XTrainScaled[test_index,:],YTrain[test_index]
  RF=RandomForestClassifier(class_weight=class_weight,max_depth=15,min_samples_leaf=10,min_samples_split=60,n_estimators=200)
  RF.fit(XTrainKF[:,[3,5,10,14,15,16,20,21]],YTrainKF.astype(int))
  YPred=RF.predict(XValKF[:,[3,5,10,14,15,16,20,21]])
  f1_score_list.append(f1_score(YValKF.astype(int),YPred))
import statistics as st
st.mean(f1_score_list)

0.8365401910513046

In [0]:
######################## Final Training Set Confusion Matrix ##################################################
RF=RandomForestClassifier(class_weight=class_weight,max_depth=15,min_samples_leaf=10,min_samples_split=60,n_estimators=200)
RF.fit(X_train[:,[3,5,10,14,15,16,20,21]],y_train)
YPred=RF.predict(X_val[:,[3,5,10,14,15,16,20,21]])
confusion_matrix(y_val,YPred)

array([[16789,  2084],
       [  814,  7345]])

In [0]:
######################### Test Set Prediction ########################################
RF=RandomForestClassifier(class_weight=class_weight,max_depth=15,min_samples_leaf=10,min_samples_split=60,n_estimators=200)
RF.fit(XTrainScaled[:,[3,5,10,14,15,16,20,21]],YTrain.astype(int))
TestPred=RF.predict(XTestScaled[:,[3,5,10,14,15,16,20,21]])
TestAdherencePrediction=[]
for item in TestPred:
  if item==1:
    TestAdherencePrediction.append('Yes')
  else:
    TestAdherencePrediction.append('No')
TestPredProbabilities=RF.predict_proba(XTestScaled[:,[3,5,10,14,15,16,20,21]])
TestPredProbabilities=TestPredProbabilities[:,1]
TestDataFrame['adherence']=TestAdherencePrediction
TestDataFrame['probability score']=TestPredProbabilities

In [0]:
TestDataFramePrediction=TestDataFrame[['patient_id','adherence','probability score']]
TestDataFramePrediction.head()

,patient_id,adherence,probability score
0,1,Yes,0.909331
1,2,Yes,0.904302
2,3,No,0.107252
3,4,No,0.135583
4,5,Yes,0.913206


In [0]:
################################### Save Test Dataset Prediction ###############################################
TestDataFramePrediction.to_csv('TestSetPrediction.csv',index=False)